In [1]:
import pyspark
from delta import *

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
24/01/10 03:48:01 WARN Utils: Your hostname, levyvix-spk-rllljjldiiq resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface eth0)
24/01/10 03:48:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/workspace/.pyenv_mirror/poetry/virtualenvs/spk-WnVPO9Fr-py3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gitpod/.ivy2/cache
The jars for the packages stored in: /home/gitpod/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-49270e82-c822-4a90-87b0-2e14f86f343f;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.0.0/delta-spark_2.12-3.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.0.0!delta-spark_2.12.jar (117ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.0.0/delta-storage-3.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.0.0!delta-storage.jar (31ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (26ms)
:: resolution report :: resolve 793ms :: ar

In [3]:
data = spark.range(0, 5)
data.write.format("delta").save("../data/test")

In [4]:
spark.read.format("delta").load("../data/test").show()

+---+
| id|
+---+
|  0|
|  1|
|  4|
|  3|
|  2|
+---+



In [5]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("../data/country_ISO_3166.csv")
)

In [6]:
df.show()

+-------------------+---------------------+---------------------+--------------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|       country_name|country_code_2_letter|country_code_3_letter|country_code_numeric|   iso_3166_2|  region|          sub_region|intermediate_region|region_code|sub_region_code|intermediate_region_code|
+-------------------+---------------------+---------------------+--------------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|        Afghanistan|                   AF|                  AFG|                   4|ISO 3166-2:AF|    Asia|       Southern Asia|               NULL|        142|             34|                    NULL|
|      Åland Islands|                   AX|                  ALA|                 248|ISO 3166-2:AX|  Europe|     Northern Europe|               NULL|        150|            154|      

In [10]:
df.write.format("delta").mode("overwrite").partitionBy("region").option(
    "overwriteSchema", "true"
).save("../data/country")